## Publication-Ready Output Tables via `Stargazer`

The [Stargazer](https://github.com/StatsReporting/stargazer) Python package support regression output for models from `PyFixest`. For details on stargazer's functionality, please take a look at the [stargazer examples](https://github.com/StatsReporting/stargazer/blob/master/examples.ipynb),

In a first step, we fit three different regression models via `PyFixest`.

In [8]:
import numpy as np
from stargazer.stargazer import LineLocation, Stargazer

import pyfixest as pf

fit1 = pf.feols("Y ~ X1 + X2", data = pf.get_data())
fit2 = pf.feols("Y ~ X1 + X2 | f1", data = pf.get_data())
fit3 = pf.feols("Y ~ X1 + X2| f1 + f2", data = pf.get_data())

We can now produce a html table with the results of the three regressions:

In [9]:
result_table = Stargazer([fit1, fit2, fit3])
result_table

But we can also ask `Stargazer` to output the results in LaTeX format:

In [10]:
print(result_table.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{3}{c}{\textit{Dependent variable: Y}} \
\cr \cline{2-4}
\\[-1.8ex] & (1) & (2) & (3) \\
\hline \\[-1.8ex]
 Intercept & 0.889$^{***}$ & & \\
& (0.108) & & \\
 X1 & -0.993$^{***}$ & -0.950$^{***}$ & -0.924$^{***}$ \\
& (0.082) & (0.067) & (0.061) \\
 X2 & -0.176$^{***}$ & -0.174$^{***}$ & -0.174$^{***}$ \\
& (0.022) & (0.018) & (0.015) \\
\hline \\[-1.8ex]
 Observations & 998 & 997 & 997 \\
 $R^2$ & 0.177 & 0.489 & 0.659 \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\
\end{tabular}
\end{table}


And we can add custom statistics!

For example, assume that we want to correct for multiple testing via the Romano-Wolf correction. We can do this as follows:

In [11]:
rwolf_res = pf.rwolf([fit1, fit2, fit3], param = "X1", seed = 123, B = 1000)
rwolf_pvalues = np.round(rwolf_res.xs("RW Pr(>|t|)"), 3).to_list()

In [12]:
rwolf_pvalues

[0.001, 0.001, 0.001]

In [13]:
result_table.add_line('X1: Romano-Wolf P-Value', rwolf_pvalues, LineLocation.FOOTER_TOP)
result_table